In [54]:
import datetime

In [8]:
# %load ../src/weather_data_clean.py
import pandas as pd

def clean_weather_data(filename):
    """Take ASOS weather data file for Stampede pass and clean it ready for input to model.
    Input: txt file
    Output: pandas dataframe
    """
    data = pd.read_csv(filename)

    # Rename two of the columns
    data.rename(columns={'valid':'date', 'tmpf':'temp'}, inplace=True)

    # Remove the few rows that have a null value for temp
    data = data[~data.temp.eq('M')]

    # Remove spaces from column names
    data.rename(columns=lambda x: x.replace(' ', ''), inplace=True)

    # Only use the standard hourly weather reading at 56 mins past each hour
    mask = data['date'].apply(lambda x: x[-2:] == '56')
    data = data[mask]

    # Create a date series to be used in the clean dataframe
    date = pd.to_datetime(data['date'])

    # Create a temp series to be used in the clean dataframe
    temp = data['temp'].apply(float)

    # Cast the null value M to zero to enable create of the raw precipitation series cast to floats
    data.p01i[data.p01i == 'M'] = 0
    raw_precipitation = data['p01i'].apply(float)

    # Create a precipitation series to be used in the clean dataframe
    precipitation = raw_precipitation.apply(lambda x: True if (x > 0) else False)

    # Convert sky coverage data to clear or cloudy and create an overcast series to be used in the clean dataframe
    sky_elements = ['skyc1', 'skyc2', 'skyc3']
    data.skyc1 = data.skyc1.astype(str)
    data.skyc2 = data.skyc2.astype(str)
    data.skyc3 = data.skyc3.astype(str)
    sky_agg = data[sky_elements].values.tolist()
    sky_reduce = [['overcast' if (('BKN' in element) or ('OVC' in element) or ('VV' in element)) else 'clear'
                    for element in row] for row in sky_agg]
    overcast = pd.Series([True if 'overcast' in row else False for row in sky_reduce])
    overcast.index = date.index

    # Cast the null value 'M' to 10.00 to enable the creation of a poor visibility series
    data.vsby[data.vsby == 'M'] = 10.00
    raw_visibility = data['vsby'].apply(float)
    poor_visibility = pd.Series([True if value < 0.50 else False for value in raw_visibility])
    poor_visibility.index = date.index

    # Cast the null value 'M' to 0 to enable the creation of a windy series
    data.sknt[data.sknt == 'M'] = 0.00
    data.gust[data.gust == 'M'] = 0.00
    wind_speed = data['sknt'].apply(float)
    gust_speed = data['gust'].apply(float)
    wind_df = pd.concat([wind_speed, gust_speed], axis=1)
    # Finally apply the function f to enable the creation of the windy column
    windy = wind_df.apply(f, axis=1)

    """Create the cleaned dataframe by concatenating the date, temp, precipitation, overcast, poor_visibility
    and windy series"""
    df = pd.concat([date, temp, precipitation, overcast, poor_visibility, windy], axis=1)
    df.columns = ['date', 'temp', 'precipitation', 'overcast', 'poor_visibility', 'windy']
    cleaned_df = df[(df['date'] > '2006-12-31') & (df['date'] < '2018-04-03')]
    return cleaned_df

def f(row):
    """Function to be able to create the windy series with windy being true if wind speed is above 10 knots
    or gust speed is above 20 knots"""
    if row['sknt'] >= 10.00:
        val = True
    elif row['gust'] >= 20.00:
        val = True
    else:
        val = False
    return val




In [49]:
weather_df = clean_weather_data('ASOS_stampede_pass/SMP-2.txt')

/Users/leanne/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [50]:
weather_df

,date,temp,precipitation,overcast,poor_visibility,windy
0,2006-12-31 23:56:00,23.00,False,True,False,False
1,2007-01-01 00:56:00,26.06,False,False,False,False
2,2007-01-01 01:56:00,24.98,False,False,False,False
3,2007-01-01 02:56:00,24.08,False,False,False,False
4,2007-01-01 03:56:00,24.08,False,False,False,False
5,2007-01-01 05:56:00,19.94,False,True,True,False
6,2007-01-01 10:56:00,19.94,True,True,False,False
7,2007-01-01 11:56:00,19.04,True,True,False,False
8,2007-01-01 12:56:00,19.94,True,True,False,False
9,2007-01-01 13:56:00,21.02,True,True,False,False


In [18]:
# %load ../src/pass_data_clean.py
import pandas as pd

def clean_pass_data(filename):
    """Take Snoqualmie pass closure data file and clean it ready for input to model.
    Input: xlsx file
    Output: Pandas dataframe
    """
    data = pd.read_excel(filename, header=[1])

    #drop unnamed/unnecessary columns
    data.drop(data.columns[[11,12,13,14]], axis=1, inplace=True)

    #drop unnecessary secondary incident columns
    data.drop(data.columns[[1,8]], axis=1, inplace=True)

    #rename 'Incident...' columns to start_time and end_time 
    data.rename(columns={'INCIDENT START TIMES FOR EACH DIRECTION':'start_time'}, inplace=True)
    data.rename(columns={'INCIDENT END TIMES - DIRECTIONAL':'end_time'}, inplace=True)

    #use only dates from 2007-01-01 to match with available weather and traffic volume data
    df = data[(data['start_time'] > '2006-12-31')]

    #rename 'Delay Time Total' to delay
    df.rename(columns={'Delay Time Total':'delay'}, inplace=True)

    #drop row with nan value in delay
    df = df.dropna(subset=['delay'])

    #create a westbound pandas series with True if westbound and false if eastbound
    westbound = pd.Series([True if value == 'WB' else False for value in df.DIRECTION])

    #create a snow pandas series with True if weather description contains sn, false otherwise
    snow = df.WEATHER.str.contains('sn', case=False, na=False, regex=True)

    #create pandas series for start and end times
    start_time = pd.to_datetime(df['start_time'])
    end_time = pd.to_datetime(df['end_time'])

    #ensure that all the pandas series created have the same index
    westbound.index = start_time.index
    snow.index = start_time.index
    end_time.index = start_time.index

    #create cleaned df with the series created
    cleaned_df = pd.concat([start_time, end_time, westbound, snow], axis=1)

    #rename columns
    cleaned_df.rename(columns={0:'westbound', 'WEATHER':'snow'}, inplace=True)

    return cleaned_df



In [19]:
pass_closure_df = clean_pass_data('Cumulative_Snoqualmie_Pass_Delay_Closures_1992_2018.xlsx')

/Users/leanne/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [20]:
pass_closure_df

,start_time,end_time,westbound,snow
830,2007-01-02 14:31:00,2007-01-02 16:45:00,False,False
831,2007-01-02 15:00:00,2007-01-02 15:44:00,True,False
832,2007-01-02 19:23:00,2007-01-02 19:33:00,True,False
833,2007-01-03 11:28:00,2007-01-03 11:40:00,True,True
834,2007-01-03 11:29:00,2007-01-03 11:46:00,False,True
835,2007-01-05 00:01:00,2007-01-05 00:16:00,False,True
836,2007-01-05 00:02:00,2007-01-05 00:11:00,True,True
837,2007-01-05 19:29:00,2007-01-05 19:51:00,False,True
838,2007-01-05 19:29:00,2007-01-05 19:44:00,True,True
839,2007-01-06 00:26:00,2007-01-06 00:46:00,False,True


In [44]:
start_end_times = list(zip(pass_closure_df.start_time, pass_closure_df.end_time))

In [45]:
start_end_times

[(Timestamp('2007-01-02 14:31:00'), Timestamp('2007-01-02 16:45:00')),
 (Timestamp('2007-01-02 15:00:00'), Timestamp('2007-01-02 15:44:00')),
 (Timestamp('2007-01-02 19:23:00'), Timestamp('2007-01-02 19:33:00')),
 (Timestamp('2007-01-03 11:28:00'), Timestamp('2007-01-03 11:40:00')),
 (Timestamp('2007-01-03 11:29:00'), Timestamp('2007-01-03 11:46:00')),
 (Timestamp('2007-01-05 00:01:00'), Timestamp('2007-01-05 00:16:00')),
 (Timestamp('2007-01-05 00:02:00'), Timestamp('2007-01-05 00:11:00')),
 (Timestamp('2007-01-05 19:29:00'), Timestamp('2007-01-05 19:51:00')),
 (Timestamp('2007-01-05 19:29:00'), Timestamp('2007-01-05 19:44:00')),
 (Timestamp('2007-01-06 00:26:00'), Timestamp('2007-01-06 00:46:00')),
 (Timestamp('2007-01-06 00:25:00'), Timestamp('2007-01-06 00:41:00')),
 (Timestamp('2007-01-06 03:42:00'), Timestamp('2007-01-06 04:55:00')),
 (Timestamp('2007-01-06 03:43:00'), Timestamp('2007-01-06 04:21:00')),
 (Timestamp('2007-01-07 08:46:00'), Timestamp('2007-01-07 10:46:00')),
 (Time

In [46]:
def get_pass_closure(date_time):
    """take a date_time and check if it is between the start and end times of a closure event
    input: datetime
    output: boolean
    """
    for row in start_end_times:
        if row[0] <= date_time <= row[1]:
            return True
    return False

In [51]:
def add_pass_closed(df):
    """take the weather df and add a new column for whether or not the pass is closed at each date_time
    input: pandas dataframe
    output: pandas dataframe
    """
    df['pass_closed'] = df['date'].map(get_pass_closure)
    return df

In [47]:
weather_df['pass_closed'] = weather_df['date'].map(get_pass_closure)

In [52]:
combined_df = add_pass_closed(weather_df)

In [53]:
combined_df

,date,temp,precipitation,overcast,poor_visibility,windy,pass_closed
0,2006-12-31 23:56:00,23.00,False,True,False,False,False
1,2007-01-01 00:56:00,26.06,False,False,False,False,False
2,2007-01-01 01:56:00,24.98,False,False,False,False,False
3,2007-01-01 02:56:00,24.08,False,False,False,False,False
4,2007-01-01 03:56:00,24.08,False,False,False,False,False
5,2007-01-01 05:56:00,19.94,False,True,True,False,False
6,2007-01-01 10:56:00,19.94,True,True,False,False,False
7,2007-01-01 11:56:00,19.04,True,True,False,False,False
8,2007-01-01 12:56:00,19.94,True,True,False,False,False
9,2007-01-01 13:56:00,21.02,True,True,False,False,False


In [55]:
def get_ordinal(date_time):
    """take a date_time and return it's ordinal value
    input: datetime
    output: ordinal
    """
    return date_time.toordinal 

In [56]:
def add_ordinal(df):
    """take the combined dataframe and add a column for the ordinal values of the date column
    input: pandas dataframe
    output: pandas dataframe
    """
    df['ordinal_date'] = df['date'].map(get_ordinal)
    return df

In [57]:
ord_df = add_ordinal(combined_df)

In [58]:
ord_df

,date,temp,precipitation,overcast,poor_visibility,windy,pass_closed,ordinal_date
0,2006-12-31 23:56:00,23.00,False,True,False,False,False,<built-in method toordinal of Timestamp object...
1,2007-01-01 00:56:00,26.06,False,False,False,False,False,<built-in method toordinal of Timestamp object...
2,2007-01-01 01:56:00,24.98,False,False,False,False,False,<built-in method toordinal of Timestamp object...
3,2007-01-01 02:56:00,24.08,False,False,False,False,False,<built-in method toordinal of Timestamp object...
4,2007-01-01 03:56:00,24.08,False,False,False,False,False,<built-in method toordinal of Timestamp object...
5,2007-01-01 05:56:00,19.94,False,True,True,False,False,<built-in method toordinal of Timestamp object...
6,2007-01-01 10:56:00,19.94,True,True,False,False,False,<built-in method toordinal of Timestamp object...
7,2007-01-01 11:56:00,19.04,True,True,False,False,False,<built-in method toordinal of Timestamp object...
8,2007-01-01 12:56:00,19.94,True,True,False,False,False,<built-in method toordinal of Timestamp object...
9,2007-01-01 13:56:00,21.02,True,True,False,False,False,<built-in method toordinal of Timestamp object...


In [65]:
d = datetime.datetime.strptime('2018-04-02 23:56:00', '%Y-%m-%d %H:%M:%S').date()

In [69]:
d

datetime.date(2018, 4, 2)

In [68]:
d_ord

<function date.toordinal>